In [16]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
from pathlib import Path
import requests
import urllib.request
import json
import pprint
from dotenv import load_dotenv
from scipy.stats import pearsonr
import numpy as np

%matplotlib inline

In [17]:
load_dotenv()
access_token = os.getenv("CQ_API_KEY")
headers = { 'Authorization': 'Bearer ' + access_token }
import urllib.request
url_a = "https://api.cryptoquant.com/v1/btc/exchange-flows/reserve?exchange=binance&window=day&from=20150101&to=20210101&limit=100000"
#only retrieves data as far back as 06/23/2017

response_a = requests.get(url_a, headers=headers)
all_exchange_reserve=response_a.json()
print(type(all_exchange_reserve))
#pprint.pprint(all_exchange_reserve)

all_exchange_df = pd.DataFrame(data=all_exchange_reserve['result']['data'])
all_exchange_df['date'] = pd.to_datetime(all_exchange_df['date'])
all_exchange_df = all_exchange_df.set_index('date')
all_exchange_df.head()

<class 'dict'>


,reserve,reserve_usd
date,,
2021-01-01,288178.322394,8.459608e+09
2020-12-31,285775.985433,8.269640e+09
2020-12-30,286856.740110,8.285127e+09
2020-12-29,285467.390927,7.816968e+09
2020-12-28,287093.528710,7.769291e+09


In [18]:
exchange_reserve_df=all_exchange_df.drop(columns=["reserve_usd"])
exchange_plot= exchange_reserve_df.hvplot(figsize=(10,5))
exchange_plot

:Curve   [date]   (reserve)

In [19]:
#Add bitcoin price to visualize

In [27]:
btc_URL = "https://api.cryptoquant.com/v1/btc/market-data/price-usd?window=day&from=20170623&to=20210101&limit=100000"
response = requests.get(btc_URL, headers=headers)
#Retrieving data as far back as 06/23/2017.
btc_price_data = response.json()
print(type(btc_price_data))

#pprint.pprint(btc_price_data)
btc_price_df = pd.DataFrame(data=btc_price_data['result']['data'])
btc_price_df = btc_price_df.drop(['price_usd_high','price_usd_low','price_usd_open'], axis = 1)
btc_price_df['date'] = pd.to_datetime(btc_price_df['date'])
btc_price_df = btc_price_df.set_index('date')

btc_price_df.head(15)


<class 'dict'>


,price_usd_close
date,
2021-01-01,29355.463333
2020-12-31,28937.490000
2020-12-30,28882.456000
2020-12-29,27383.052000
2020-12-28,27061.882000
2020-12-27,26269.252000
2020-12-26,26481.480000
2020-12-25,24709.222000
2020-12-24,23738.966000


In [22]:
#combine btc price and btc reserve on exchanges graphs

In [23]:
btc_plot=btc_price_df.hvplot()
btc_plot*exchange_plot


:Overlay
   .Curve.I  :Curve   [date]   (price_usd_close)
   .Curve.II :Curve   [date]   (reserve)

In [24]:
#correlation
covariance = np.cov(all_exchange_df['reserve'],btc_price_df['price_usd_close'])
corr, _ = pearsonr(all_exchange_df['reserve'], btc_price_df['price_usd_close'])
print('Pearsons correlation: %.3f' % corr)
#result is they are not heavily correlated

Pearsons correlation: 0.322


In [25]:
def get_exchange_reserves(start, end):
    load_dotenv()
    access_token = os.getenv("CQ_API_KEY")
    headers = { 'Authorization': 'Bearer ' + access_token }
    import urllib.request
    url_a = f"https://api.cryptoquant.com/v1/btc/exchange-flows/reserve?exchange=binance&window=day&from={start}&to={end}&limit=100000"
#only retrieves data as far back as 06/23/2017

    response_a = requests.get(url_a, headers=headers)
    all_exchange_reserve=response_a.json()
    print(type(all_exchange_reserve))
#pprint.pprint(all_exchange_reserve) to see it better

    all_exchange_df = pd.DataFrame(data=all_exchange_reserve['result']['data'])
    all_exchange_df['date'] = pd.to_datetime(all_exchange_df['date'])
    all_exchange_df = all_exchange_df.set_index('date')
    exchange_reserve_df=all_exchange_df.drop(columns=["reserve_usd"])

    
    return exchange_reserve_df

In [26]:
test_df = get_exchange_reserves(20200101, 20210101)
test_df.head()

<class 'dict'>


,reserve
date,
2021-01-01,288178.322394
2020-12-31,285775.985433
2020-12-30,286856.740110
2020-12-29,285467.390927
2020-12-28,287093.528710
